In [6]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
import plotly.graph_objects as go

In [7]:
df = pd.read_csv('Moscow_2021.csv', sep=';', parse_dates=['2021'], dayfirst=True, index_col='2021')
df = df.drop(['Length', 'Change'], axis=1)
df.loc[:, ['AT_east', 'AT_west', 'NT_east', 'NT_west']] = df.loc[:, ['AT_east', 'AT_west',
                                                                    'NT_east', 'NT_west']].mask(df == '?', None)
df.head(2)

,Sunrise,Noon,Sunset,AT_east,NT_east,CT_east,CT_west,NT_west,AT_west
2021,,,,,,,,,
2021-01-01,9:00:29,12:34:13,16:07:58,6:40:27,7:25:37,8:14:03,16:54:24,17:42:50,18:28:00
2021-01-02,9:00:12,12:34:41,16:09:10,6:40:24,7:25:31,8:13:52,16:55:29,17:43:51,18:28:58


In [8]:
df_time = df.copy()
for column in df_time.columns:
    df_time[column] = pd.to_datetime(df_time[column], format='%H:%M:%S').dt.time
df_time.head(2)

,Sunrise,Noon,Sunset,AT_east,NT_east,CT_east,CT_west,NT_west,AT_west
2021,,,,,,,,,
2021-01-01,09:00:29,12:34:13,16:07:58,06:40:27,07:25:37,08:14:03,16:54:24,17:42:50,18:28:00
2021-01-02,09:00:12,12:34:41,16:09:10,06:40:24,07:25:31,08:13:52,16:55:29,17:43:51,18:28:58


In [9]:
dfn = df.copy()
for column in dfn.columns:
    dfn[column] = pd.to_numeric(dfn[column].apply(pd.Timedelta))*1.6666666666667e-11
dfn = dfn.mask(dfn < 0, None)
dfn = dfn[['AT_east', 'NT_east', 'CT_east', 'Sunrise', 'Noon', 'Sunset', 'CT_west', 'NT_west', 'AT_west']]
dfn.head()

,AT_east,NT_east,CT_east,Sunrise,Noon,Sunset,CT_west,NT_west,AT_west
2021,,,,,,,,,
2021-01-01,400.45,445.616667,494.050000,540.483333,754.216667,967.966667,1014.400000,1062.833333,1108.0
2021-01-02,400.4,445.516667,493.866667,540.200000,754.683333,969.166667,1015.483333,1063.85,1108.966667
2021-01-03,400.283333,445.366667,493.650000,539.850000,755.133333,970.416667,1016.616667,1064.9,1109.983333
2021-01-04,400.133333,445.15,493.350000,539.433333,755.583333,971.733333,1017.800000,1066.0,1111.033333
2021-01-05,399.933333,444.9,493.016667,538.950000,756.016667,973.100000,1019.033333,1067.133333,1112.116667


In [10]:
x = dfn.index
y_m = [1440]*len(x)
y_c = ['23:59:59']*len(x)

fig = go.Figure()
for column in dfn.columns[2:7]:
    fig.add_trace(go.Scatter(name=column, x=x, y=dfn[column], customdata=df_time[column],
               hovertemplate='%{customdata}', fill='tonexty', mode='none'))
fig.add_trace(go.Scatter(name='Midnight', x=x, y=y_m, hoverinfo='skip',
                         showlegend=False, fill='tonexty', mode='none'))

fig.update_layout(hovermode='x',
                  title = {'text': 'Moscow 2021', 'x': .5},
                  legend_traceorder='normal',
#                   yaxis_range=[1440, 0],
                 )
fig.update_xaxes(tickvals=['2021-01-01', '2021-02-01','2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
                           '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01','2021-11-01', '2021-12-01'],
                 ticktext=['Jan', 'Feb', 'Mar', 'April', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                 side='top',
                )
fig.update_yaxes(range=[1440, 0],
                 tickvals=np.arange(0, 1440, 120),
                 ticktext=pd.date_range('00:00','23:59', freq='120min').time)
fig.show()